#A simple, yet robust search engine in Python

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)


C:\Users\Xingy\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [11]:
import numpy as np

In [2]:
###link to data:
#https://drive.google.com/file/d/1pJFPa5772JiXWxZ9pGpwNbO6D0BBCEXZ/view?usp=sharing
df = pd.read_excel('context.xlsx')

### Preprocess and tokenise

In [4]:
!pip install rank_bm25

In [31]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm

  ERROR: HTTP error 404 while getting https://github.com/explosion/spacy-models/releases/download/en_core_web_sm
  ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/en_core_web_sm because of error 404 Client Error: Not Found for url: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm
ERROR: Could not install requirement https://github.com/explosion/spacy-models/releases/download/en_core_web_sm because of HTTP error 404 Client Error: Not Found for url: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm for URL https://github.com/explosion/spacy-models/releases/download/en_core_web_sm


In [4]:
import spacy
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(df.context.str.lower().values, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)


19035it [00:21, 893.95it/s]


### Simple BM25 search

In [5]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tok_text)

In [35]:
query="Beyonce Destiny's"
tokenized_query = query.lower().split(" ")
import time

t0 = time.time()
results = bm25.get_top_n(tokenized_query, df.context.values, n=3)
t1 = time.time()
print(f'Searched 19,000 records in {round(t1-t0,3) } seconds \n')
for i in results:
  print(i)

Searched 19,000 records in 0.056 seconds 

In August, the couple attended the 2011 MTV Video Music Awards, at which Beyoncé performed "Love on Top" and started the performance saying "Tonight I want you to stand up on your feet, I want you to feel the love that's growing inside of me". At the end of the performance, she dropped her microphone, unbuttoned her blazer and rubbed her stomach, confirming her pregnancy she had alluded to earlier in the evening. Her appearance helped that year's MTV Video Music Awards become the most-watched broadcast in MTV history, pulling in 12.4 million viewers; the announcement was listed in Guinness World Records for "most tweets per second recorded for a single event" on Twitter, receiving 8,868 tweets per second and "Beyonce pregnant" was the most Googled term the week of August 29, 2011.
In September 2010, West wrote a series of apologetic tweets addressed to Swift including "Beyonce didn't need that. MTV didn't need that and Taylor and her family fr

# Fast text model

In [9]:
from gensim.models.fasttext import FastText

ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    size=100, # embedding dimension (default)
    window=10, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=5, # only consider tokens with at least n occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    min_n=2, # min character n-gram
    max_n=5 # max character n-gram
)
ft_model.build_vocab(tok_text) # tok_text is our tokenized input text - a list of lists relating to docs and tokens respectivley

ft_model.train(
    tok_text,
    epochs=6,
    total_examples=ft_model.corpus_count, 
    total_words=ft_model.corpus_total_words)

ft_model.save('_fasttext.model') #save
ft_model = FastText.load('_fasttext.model') #load

In [13]:
import pickle

# Create weighted vector

In [14]:
weighted_doc_vects = []

for i,doc in tqdm(enumerate(tok_text)):
    doc_vector = []
    for word in doc:
        vector = ft_model[word]
        weight = (bm25.idf[word] * ((bm25.k1 + 1.0)*bm25.doc_freqs[i][word])) / (bm25.k1 * (1.0 - bm25.b + bm25.b *(bm25.doc_len[i]/bm25.avgdl))+bm25.doc_freqs[i][word])
        weighted_vector = vector * weight
        doc_vector.append(weighted_vector)
    doc_vector_mean = np.mean(doc_vector,axis=0)
    weighted_doc_vects.append(doc_vector_mean)
    pickle.dump( weighted_doc_vects, open( "weighted_doc_vects.p", "wb" ) ) #save the results to disc


0it [00:00, ?it/s]C:\Users\Xingy\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
19035it [1:22:01,  3.87it/s] 


In [16]:
!pip install nmslib

In [17]:
import nmslib

# create a matrix from our document vectors
data = np.vstack(weighted_doc_vects)

# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

In [42]:
input = "Beyoncé actor".lower().split()

query = [ft_model[vec] for vec in input]
query = np.mean(query,axis=0)

t0 = time.time()
ids, distances = index.knnQuery(query, k=10)
t1 = time.time()
print(f'Searched {df.shape[0]} records in {round(t1-t0,4) } seconds \n')
for i,j in zip(ids,distances):
  print(round(j,2))
  print(df.context.values[i] +  '    ContextID=  '+ str(df.contextID.values[i]))

Searched 19035 records in 0.0 seconds 

0.24
The feminism and female empowerment themes on Beyoncé's second solo album B'Day were inspired by her role in Dreamgirls and by singer Josephine Baker. Beyoncé paid homage to Baker by performing "Déjà Vu" at the 2006 Fashion Rocks concert wearing Baker's trademark mini-hula skirt embellished with fake bananas. Beyoncé's third solo album I Am... Sasha Fierce was inspired by Jay Z and especially by Etta James, whose "boldness" inspired Beyoncé to explore other musical genres and styles. Her fourth solo album, 4, was inspired by Fela Kuti, 1990s R&B, Earth, Wind & Fire, DeBarge, Lionel Richie, Teena Marie with additional influences by The Jackson 5, New Edition, Adele, Florence and the Machine, and Prince.    ContextID=  40
0.25
Beyoncé's first solo recording was a feature on Jay Z's "'03 Bonnie & Clyde" that was released in October 2002, peaking at number four on the U.S. Billboard Hot 100 chart. Her first solo album Dangerously in Love was rel

C:\Users\Xingy\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
